In [4]:
import serpentTools
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
#Graphing of K-eigenvalue

#this is to read _res.m output files from serpent and extract K-eigenvalues
import glob
import matplotlib.pyplot as plt
import numpy as np
import os

#set file path and file type to look for
path = "/Users/alessandroingegno/Desktop/NE170/ARG/Outputs/Final/*_res.m"


def file_read(file):

    with open(file,'r') as output:
    
        lines = output.readlines() #reading output file
        search = "% Criticality eigenvalues:" #keyword to search for
        search2 = "% Analog reaction rate estimators:"
        indicies = [] #array of indicies of search word
        indicies2 = []

        #looping through lines and finding index of search word
        for i in range(len(lines)):
            if lines[i].find(search) != -1:
                indicies.append(i)
            elif lines[i].find(search2) != -1:
                indicies2.append(i)
            else:
                continue
    
    #returning file as a list and the index of search word
    return lines, indicies, indicies2

def output_processing(lines, index):
    #extracting different k-eigenvalues per index
    keffs = lines[index+2:index+4]

    values = [] #k-eingenvalues per index
    errors = [] #standard deviation of the eigenvalues

    #adjusting string to return only eigenvalue and standard deviation
    for keff in keffs:
        values.append(float(keff[47:58]))
        errors.append(float(keff[59:66]))
    
    return values, errors

def conversion_ratio(lines,index):
    CR = lines[index+2]

    value = float(CR[47:58])
    error = float(CR[59:66])

    return value, error

def results(path):

    files = glob.glob(path) #getting files from path
    files = sorted(files)
    display(files)
    #figure for plotting
    fig = plt.figure()
    ax = plt.axes()
    plt.grid()
    #plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.plasma(np.linspace(0,1,len(files))))

    
    #Looping through files in outputs
    j = 0
    for file in files:

        #reading lines and getting keff index
        lines, indicies, indicies2 = file_read(file)

        #lists for storing each burnup stage keff
        ana_keff = []
        imp_keff = []

        ana_err = []
        imp_err = []

        cr = []
        cr_err = []

        for i in indicies:
            keffs, sd = output_processing(lines,i)
            ana_keff.append(keffs[0])
            ana_err.append(sd[0])

            imp_keff.append(keffs[-1])
            imp_err.append(sd[-1])
        
        for i in indicies2:
            value, error = conversion_ratio(lines,i)

            cr.append(value)
            cr_err.append(error)

        ana = np.array(ana_keff)
        anaerr = np.array(ana_err)
        imp = np.array(imp_keff)
        imperr = np.array(imp_err)

        cr = np.array(cr)
        cr_err = np.array(cr_err)

        print(ana_keff,ana_err)
        print(cr, cr_err)
        #print(imp_keff,imp_err)

        #burnup_days = [0,1460,2920,4380,5840,7300]
        burnup_days = np.array([0,10, 50, 90, 365, 730, 1095, 1460, 1825, 2190, 2555, 2920, 3285, 3650, 4015, 4380, 4745, 5110, 5474, 5840, 6205, 6570, 6935, 7300])/365.25
        #print(str(file)[-10:-6])

        #label = str(float(str(file)[-10:-6])/1000) + " P/D"

        #nums = [143.83,148.83,163.83,138.83,128.83,135.83,173.83]
        nums = ['Analog Keff', "Conversion Ratio"]
        ana_max = ana + anaerr
        ana_min = ana - anaerr

        imp_max = imp + imperr
        imp_min = imp - imperr

        cr_max = cr + cr_err
        cr_min = cr - cr_err

        #ax.plot(burnup_days,ana_keff,label=nums[j],color='black',linewidth=0.75)
        #ax.fill_between(burnup_days,ana_min,ana_max, alpha=0.2,color="black")
        #j+=1

        #ax.plot(burnup_days,cr,label=nums[j],color='Red',linewidth=0.75)
        #ax.fill_between(burnup_days,cr_min,cr_max, alpha=0.2,color='Red')
    
        #j+=1
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [1, 1]})

# Plot the first line on the upper axis
    ax1.plot(burnup_days, ana, label='Analog Keff',linewidth=0.75,color='black')
    ax1.fill_between(burnup_days,ana_min,ana_max, alpha=0.2,color="black")
    ax1.set_ylim(1.002, 1.007)  # Set desired scale for the upper axis
# Plot the second line on the lower axis
    ax2.plot(burnup_days, cr, label='Conversion Ratio', color='red',linewidth=0.75)
    ax2.fill_between(burnup_days,cr_min,cr_max, alpha=0.2,color='Red')
    ax2.set_ylim(0.81, 0.84)  # Set desired scale for the lower axis

# Indicate the break in the y-axis
    ax1.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax1.xaxis.tick_top()
    ax2.xaxis.tick_bottom()
    d = 0.02  # How big to make the diagonal lines in axes coordinates
    kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
    ax1.plot((-d, +d), (-d, +d), **kwargs)        # Top diagonal
    ax1.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # Top diagonal
    kwargs.update(transform=ax2.transAxes)
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # Bottom diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # Bottom diagonal

# Add labels and legend
    ax1.set_ylabel('Keff')
    ax2.set_ylabel('Conversion Ratio')
    ax2.set_xlabel('Burnup Time [Years]')
    fig.tight_layout()
    ax1.legend()
    ax2.legend(loc='lower right')
    #plt.show()
    #fig.suptitle('Keff Over 20 Year Burnup (124 Assemblies)')
    #ax.set(xlabel="Burn Up Time [Years]",ylabel="Keff",title="Keff Over 20 Year Burnup (124 Assemblies)",ylim=(0.999,1.01))
    #plt.legend(loc="lower right")
    #plt.savefig("keff_CR",dpi=600,bbox_inches='tight')
    
results(path)